In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# 加载MNIST数据集
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data

# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.2, random_state=42)

/usr/local/lib/python3.9/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
from sklearn.decomposition import PCA
pca = PCA(n_components=154)
X_reduced = pca.fit_transform(X_train)
X_recoverded = pca.inverse_transform(X_reduced)

In [ ]:
rnd_pca = PCA(n_components=154,svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

# 8.3.9增量pca

In [ ]:
from sklearn.decomposition import IncrementalPCA
import numpy as np
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train,n_batches):
  inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

In [3]:
import platform
import psutil

# 打印CPU信息
print("CPU型号：", platform.processor())
print("CPU核心数：", psutil.cpu_count(logical=True))
print("CPU主频：", psutil.cpu_freq().max, "MHz")


CPU型号： x86_64
CPU核心数： 2
CPU主频： 0.0 MHz


In [2]:
import tensorflow as tf

# 查看GPU设备及其内存信息
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            gpu_name = gpu.name
            print(f"GPU 设备名称：{gpu_name}")
            gpu_memory = tf.config.experimental.get_memory_info(gpu)
            print(f"GPU 可用内存: {gpu_memory.available / 1024 /1024:.2f} MB")
            print(f"GPU 总内存: {gpu_memory.total / 1024 /1024:.2f} MB")
            print(f"GPU 内存使用率: {gpu_memory.percent}%\n")

    except RuntimeError as e:
        print('无法查询GPU内存：', e)
else:
    print("没有找到GPU设备")


GPU 设备名称：/physical_device:GPU:0


TypeError: ignored

In [ ]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2,kernel="rbf",gamma=0.04)
X_reduced = rbf_pca.fit_transform(X_train)

In [3]:
# 将数据集转换为TensorFlow张量
import tensorflow as tf
import tensorflow as tf
from sklearn.datasets import make_circles
from sklearn.decomposition import KernelPCA
X_train = tf.constant(X, dtype=tf.float32)

# 使用内核PCA进行降维
with tf.device('/GPU:0'):  # 将操作放在GPU上执行
    rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
    X_reduced = rbf_pca.fit_transform(X_train)

# 打印降维后的数据
print(X_reduced)

NameError: ignored

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.datasets import make_circles
from sklearn.decomposition import KernelPCA

# 生成数据集
# X, _ = make_circles(n_samples=1000, noise=0.1, random_state=42)

# 将数据集转换为TensorFlow张量
dataset = tf.data.Dataset.from_tensor_slices(tf.constant(X_train, dtype=tf.float32))

# 将TensorFlow张量转换为NumPy数组
X_np = np.vstack(list(dataset.as_numpy_iterator()))

# 使用内核PCA进行降维
with tf.device('/GPU:0'):  # 将操作放在GPU上执行
    rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
    X_reduced = rbf_pca.fit_transform(X_np)

# 打印降维后的数据
print(X_reduced)


In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca",KernelPCA(n_components=2)),
    ("log_reg",LogisticRegression())
])
param_grid = [{
    "kpca__gamma":np.linspace(0.03,0.05,10),
    "kpca__kernel":["rbf","sigmoid"]
}]

grid_search = GridSearchCV(clf,param_grid,cv=3)
grid_search.fit(X,y)

NameError: ignored